In [199]:
import numpy as np
import sympy as sym
import control as ctrl
from matplotlib import pyplot as plt

In [200]:
g=9.81  # m/s^2
L=0.125 # distance from pendulum's centre of mass to pivot in m
a=0.016 # radius of pulley in m
m=0.32  # mass of pendulum in kg
M=0.7  # mass of carriage in kg
I=8e-5  # moment of inertia on motor shaft in kg m^2
km = 0.08  # torque motor constant in Nm/A
ka = -0.50 # amplifier constant in A/V

gamma =  M/m + I/(m*np.power(a,2))

In [201]:

omega_1 = sym.symbols('omega_1')
omega_0 = sym.symbols('omega_0')

Ac = sym.Matrix(4,4, [0,1,0,0,
                        0,0,omega_1**2 - omega_0**2, 0,
                        0,0,0,1,
                        0,0,-omega_0**2, 0])

B = sym.Matrix(4,1, [0,1,0,1])
Sc=np.diag([-1/12.5, -1/2.23, L/3.18, L/0.64])
opamp_c = np.diag([-20,-30, 20, -10]) # for crane controller
Cc=-(ka*km/(m*a*gamma))*np.linalg.solve(Sc,opamp_c)

s = sym.symbols('s')
t = sym.symbols('t')

k1,k2,k3,k4 = sym.symbols('k1 k2 k3 k4')

K =  sym.Matrix(1,4, [k1,k2,k3,k4])

cG_ = sym.eye(4)*s - Ac
Gc_sys = Cc * cG_.inv() * B

cG_ = sym.eye(4)*s - Ac + B*K
crane_poly = sym.poly(sym.det(cG_), s)
Gc_closed = Cc * cG_.inv() * B

#xc = sympy.inverse_laplace_transform(Gc / s, s, t)

# factorise and convert to latex
print("Closed loop characteristic polynomial for crane model:")
print(sym.latex(crane_poly))

print("Closed loop transfer function for crane model:")
print(sym.latex(sym.simplify(Gc_closed)))

Closed loop characteristic polynomial for crane model:
\operatorname{Poly}{\left( s^{4} + \left(k_{2} + k_{4}\right) s^{3} + \left(k_{1} + k_{3} + \omega_{0}^{2}\right) s^{2} + k_{2} \omega_{1}^{2} s + k_{1} \omega_{1}^{2}, s, domain=\mathbb{Z}\left[k_{1}, k_{2}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right] \right)}
Closed loop transfer function for crane model:
\left[\begin{matrix}\frac{617.283950617284 \left(\omega_{1}^{2} + s^{2}\right)}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\\frac{165.185185185185 s \left(\omega_{1}^{2} + s^{2}\right)}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\\frac{1256.2962962963 s^{2}}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\- \frac{126.41975308642 s^{3}}{k_{1} \omega_{1}^{2} + k_{1} 

In [202]:

Ap = sym.Matrix(4,4, [0,1,0,0,
                        0,0,omega_0**2 - omega_1**2, 0,
                        0,0,0,1,
                        0,0,omega_0**2, 0])
opamp_p = np.diag([10,20,30,-20]) # for inverted pendulum controller
Sp=np.diag([-1/12.5, -1/2.23, L/3.18, -L/0.64])
Cp=-(ka*km/(m*a*gamma))*np.linalg.solve(Sp,opamp_p)

pG_ = sym.eye(4)*s - Ap
Gp_sys = Cp * pG_.inv() * B

pG_ = sym.eye(4)*s - Ap + B*K
poly = sym.poly(sym.det(pG_), s)

Gp_closed = Cc * pG_.inv() * B

#xc = sym.inverse_laplace_transform(Gp_closed / s, s, t)

# factorise and convert to latex
print("Closed loop characteristic polynomial for crane model:")
print(sym.latex(poly))

print("Closed loop transfer function for crane model:")
print(sym.latex(sym.simplify(Gp_closed)))

Closed loop characteristic polynomial for crane model:
\operatorname{Poly}{\left( s^{4} + \left(k_{2} + k_{4}\right) s^{3} + \left(k_{1} + k_{3} - \omega_{0}^{2}\right) s^{2} -  k_{2} \omega_{1}^{2} s -  k_{1} \omega_{1}^{2}, s, domain=\mathbb{Z}\left[k_{1}, k_{2}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right] \right)}
Closed loop transfer function for crane model:
\left[\begin{matrix}\frac{617.283950617284 \left(- \omega_{1}^{2} + s^{2}\right)}{- k_{1} \omega_{1}^{2} + k_{1} s^{2} - k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} - \omega_{0}^{2} s^{2} + s^{4}}\\\frac{165.185185185185 s \left(- \omega_{1}^{2} + s^{2}\right)}{- k_{1} \omega_{1}^{2} + k_{1} s^{2} - k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} - \omega_{0}^{2} s^{2} + s^{4}}\\\frac{1256.2962962963 s^{2}}{- k_{1} \omega_{1}^{2} + k_{1} s^{2} - k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} - \omega_{0}^{2} s^{2} + s^{4}}\\- \frac{126.41975308642 s^{3}}{- k_{1} \omega_{1